In [1]:
import numpy as np
import pandas as pd
import json

In [4]:
netflixData = pd.read_csv('/Users/noufalzamel/Desktop/JSON/netflix_titles_MONGODB.csv')
directors_df = pd.read_csv('/Users/noufalzamel/Desktop/JSON/directors.csv',sep=';')
cast_df = pd.read_csv('/Users/noufalzamel/Desktop/JSON/cast.csv',sep=';')
reviews_data = pd.read_csv('/Users/noufalzamel/Desktop/reviews.csv')

In [5]:
netflixData.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [117]:
directors_df.head(2)

,director_id,director_firstName,director_lastName
0,1,Richard,Finn
1,2,Tim,Maltby


In [118]:
#Adding director ids in netflixData dataset by matching the name
directors_df['director_fullName']=directors_df['director_firstName']+" "+directors_df['director_lastName']
netflixData['director_id'] = (netflixData['director'].str.split(',\s*')
    .explode()
    .map(directors_df.set_index('director_fullName')['director_id'])
    .groupby(level=0).agg(list)
)


In [119]:
#Adding cast ids in netflixData dataset by matching the name
cast_df['cast_fullName']=cast_df['cast_firstName']+" "+cast_df['cast_lastName']
netflixData['cast_id'] = (netflixData['cast'].str.split(',\s*')
    .explode()
    .map(cast_df.set_index('cast_fullName')['cast_id'])
    .groupby(level=0).agg(list)
)


In [120]:
netflixData.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,director_id,cast_id
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,"[1.0, 2.0]","[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
1,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,[3.0],"[29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36...."


In [121]:
## converting ids to int
netflixData['director_id']=netflixData['director_id'].apply(lambda x: [int(v) for v in x if not np.isnan(v)])
netflixData['cast_id']=netflixData['cast_id'].apply(lambda x: [int(v) for v in x if not np.isnan(v)])
netflixData.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,director_id,cast_id
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,"[1, 2]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
1,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,[3],"[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]"


In [122]:
## dropping director and cast names columns from netflixData dataset
netflixData=netflixData.drop(['director','cast'],axis=1)

In [123]:
## dropping tomato_critic_review column as it is all null
reviews_data=reviews_data.drop(['tomato_critic_review'],axis=1)


In [128]:
reviews_data['imdb_user_review']=reviews_data['imdb_user_review'].fillna(-999)
reviews_data['imdb_critic_review']=reviews_data['imdb_critic_review'].fillna(-999)
reviews_data['tomato_user_review']=reviews_data['tomato_user_review'].fillna(-999)

reviews_data.head(2)

,Unnamed: 0,title,imdb_user_review,imdb_critic_review,tomato_user_review
0,0,Norm of the North: King Sized Adventure,3.1,-999.0,36.0
1,1,#REALITYHIGH,5.2,-999.0,-999.0


In [135]:
## Mapping imdb_user_review
mapper1 = reviews_data.set_index('title')['imdb_user_review'].to_dict()
netflixData['imdb_user_review']=netflixData['title'].map(mapper1)

## Mapping imdb_critic_review
mapper2 = reviews_data.set_index('title')['imdb_critic_review'].to_dict()
netflixData['imdb_critic_review']=netflixData['title'].map(mapper2)

## Mapping tomato_user_review
mapper3 = reviews_data.set_index('title')['tomato_user_review'].to_dict()
netflixData['tomato_user_review']=netflixData['title'].map(mapper3)



In [136]:
## export netflixData dataset as json file
jsonData=netflixData.to_dict(orient='records')
with open('ShowsData.json','w',encoding='utf-8') as outfile:
     json.dump(jsonData, outfile) 


In [15]:
## export cast dataset as json file
cast_df=cast_df.drop(['cast_fullName'],axis=1)
jsonCastData=cast_df.to_dict(orient='records')
with open('castData.json','w',encoding='utf-8') as outfile:
     json.dump(jsonCastData, outfile) 


In [17]:
## export directors dataset as json file
directors_df=directors_df.drop(['director_fullName'],axis=1)
jsonDirectorData=directors_df.to_dict(orient='records')
with open('directorData.json','w',encoding='utf-8') as outfile:
     json.dump(jsonDirectorData, outfile) 
